In [1]:
import pandas as pd
from olist.order import Order
from olist.data import Olist
from olist.utils import haversine_distance

In [2]:
olist = Olist().get_data()
orders = olist['orders']
sellers = olist['sellers']
items = olist['order_items']
geolocations = olist['geolocation']
customers = olist['customers']

: 

In [3]:
#Selectionner que les colonnes dont on aura besoin
orders_df=orders[['order_id','customer_id']]
sellers_df=sellers[['seller_id','seller_zip_code_prefix']]
items_df=items[['order_id','seller_id']]
geolocations_df=geolocations[['geolocation_zip_code_prefix','geolocation_lat','geolocation_lng']].drop_duplicates()

# Joindre les DataFrames sur les colonnes communes
customers_geolocation = customers.merge(geolocations_df, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
sellers_geolocation = sellers_df.merge(geolocations_df, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')
#Fusionner items avec les informations des vendeurs et clients géolocalisés
merged_data2=orders_df.merge(customers_geolocation, on='customer_id')
merged_data = items_df.merge(sellers_geolocation, on='seller_id').merge(merged_data2, on='order_id')
# Calculer la distance directement dans la méthode apply
merged_data['distance_seller_customer'] = merged_data.apply(lambda row: haversine_distance(
   row['geolocation_lng_customer'], row['geolocation_lat_customer'], row['geolocation_lng_seller'], row['geolocation_lat_seller']), axis=1)
# Créer une nouvelle DataFrame avec les distances par order_id
distance_df = merged_data[['order_id', 'distance_seller_customer']]